In [1]:
import sys
import importlib

from langchain.chat_models.openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.tools import Tool

# quickfix, since modules from relative imports were not found
sys.path.append("/Users/stefanie/Documents/code/RAG_Langchain")
importlib.import_module("modules.prompts", package="RAG_Langchain")
importlib.import_module("modules.agent", package="RAG_Langchain")

from modules.prompts import DebugPrompt, MobyPrompt
from modules.retriever import Retriever

In [2]:
# define user query
query = "Find a sentence that would be terrible advice for a DIY project."

In [3]:
# get additional context from embedding retriever
retriever = Retriever(query)
retriever = retriever.embed().retrieve()

Created a chunk of size 4330, which is longer than the specified 2500
Created a chunk of size 2797, which is longer than the specified 2500
Created a chunk of size 2594, which is longer than the specified 2500
Created a chunk of size 2872, which is longer than the specified 2500
Created a chunk of size 2682, which is longer than the specified 2500
Created a chunk of size 2953, which is longer than the specified 2500
Created a chunk of size 2597, which is longer than the specified 2500
Created a chunk of size 3670, which is longer than the specified 2500
Created a chunk of size 2542, which is longer than the specified 2500
Created a chunk of size 2881, which is longer than the specified 2500
Created a chunk of size 3525, which is longer than the specified 2500
Created a chunk of size 2577, which is longer than the specified 2500
Created a chunk of size 2757, which is longer than the specified 2500
Created a chunk of size 2514, which is longer than the specified 2500
Created a chunk of s

In [4]:
# get prompt
prompt = MobyPrompt.prompt
#prompt = DebugPrompt.prompt

In [5]:
# define llm model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_stop = llm.bind(stop=["\nQuestion:"])

In [6]:
# legacy way
llmchain = LLMChain(llm=llm_with_stop, prompt=prompt, output_key='output', verbose=True)
resp = llmchain({'context': retriever.context, 'query': query})
print(resp['output'])



> Entering new LLMChain chain...
Prompt after formatting:
Human: We are playing the party game "Bring your own Book". You are only allowed to answer 
    with anything else than quotes from the provided documents, known to you as "context". 
    
    "context" contains passages from Herman Melville's "Moby-Dick" and you can use your knowledge 
    about this novel and the historical setting to find the most suitable anwers. Please take care
    that your answer matches correct text type requested in the query.

    Only answer with direct quotes from the "context".

    context = A belaying pin is found too large to be easily inserted into its hole:
the carpenter claps it into one of his ever-ready vices, and
straightway files it smaller. A lost land-bird of strange plumage
strays on board, and is made a captive: out of clean shaved rods of
right-whale bone, and cross-beams of sperm whale ivory, the carpenter
makes a pagoda-looking cage for it. An oarsman sprains his wrist: the
carpe

In [7]:
# LCEL I
chain = ({"context": RunnablePassthrough(), "query": RunnablePassthrough()}) | prompt | llm | StrOutputParser()

chain.invoke({"context": retriever.context,
              "query": query})


'"Turn not thy back to the compass; accept the first hint of the hitching tiller; believe not the artificial fire, when its redness makes all things look ghastly."'

In [8]:
# LCEL II
chain = ({"context": retriever.retriever, "query": RunnablePassthrough()}) | prompt | llm | StrOutputParser()

chain.invoke(query)

'A belaying pin is found too large to be easily inserted into its hole: the carpenter claps it into one of his ever-ready vices, and straightway files it smaller.'

In [9]:
# define tools
tools = [
    Tool.from_function(
        func=retriever.retrieve,
        name="DocAgent",
        description="useful for when you need to search for answer from documents",
    ),
]

In [10]:
# define langchain agent
from langchain.agents import initialize_agent, AgentType
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         agent_scratchpad="intermediate_steps",
                         verbose=True)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["history"],
    }
    | prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

In [11]:
# add memory
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)

In [12]:
# define agent executer
from langchain.agents.agent import AgentExecutor
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, memory=memory, handle_parsing_errors=True
)

In [13]:
agent_executor.invoke({"input": query})['output']



> Entering new AgentExecutor chain...


KeyError: 'context'

In [ ]:
# run testsuite
pass